In [1]:
import pandas as pd
import numpy as np

In [2]:
def process_activity_column(column):
    change_str = ""
    changing = False
    processed_column = column.copy()
    max_stack = [0]
    max_stack_index = []
    for index, state in enumerate(column):
        if str(state).split()[-1] == 'begin':
            changing = True
#             print("Begin Found")
            if not change_str == '':# and (state.strip().split()[0] not in change_str.split('-')):
                change_str += '-'+ state.strip().split()[0]
            else:
                change_str = state.strip().split()[0]
#             print(index, change_str)
        if changing:
            processed_column[index] = change_str.split('-')
            if len(processed_column[index]) > max_stack[0]:
                max_stack = [len(processed_column[index])]
                max_stack_index = [index]
            elif len(processed_column[index]) == max_stack[0]:
                max_stack.append(len(processed_column[index]))
                max_stack_index.append(index)
        if str(state).split()[-1] == 'end':
            processed_column[index] = change_str.split('-')
#             print("End Found")
            end_act = state.strip().split()[0]
#             print("End act: ", end_act)
            acts = change_str.split('-')
            acts.remove(end_act)
            if not acts:
#                 print("no acts")
                changing = False
            change_str = '-'.join(acts)
#             print(index, acts, change_str)
    return processed_column, max_stack, max_stack_index

In [3]:
data_milan = pd.read_csv("dataset/milan/data_test", sep = "\t", index_col=0, parse_dates=[0], infer_datetime_format=True, skip_blank_lines = True, names = ['Timestamp (UTC)', 'sensorID', 'state', 'activity'])
data_milan = data_milan.sort_index()
processed_column, max_stack, max_stack_index = process_activity_column(list(data_milan['activity']))
data_milan['mod_col'] = processed_column
data_milan['mod_col'] = data_milan['mod_col'].fillna('None')

In [4]:
print(max_stack, max_stack_index)
print(data_milan.iloc[2661])
print(data_milan.index[0] + pd.Timedelta(minutes=15))

[2, 2, 2, 2, 2, 2] [2661, 2662, 2663, 2664, 2665, 2666]
sensorID                              M001
state                                   ON
activity                  Leave_Home begin
mod_col     [Kitchen_Activity, Leave_Home]
Name: 2009-10-16 14:49:54.000046, dtype: object
2009-10-16 00:16:04.000059


In [5]:
def process_subset(df, uni_list_beg, cols):
    act_col = df['mod_col'].tolist()
    uni_list = []
    for i in act_col:
        uni_list += ['Other'] if i == 'None' else i
    uni_list = set(uni_list)
    act_col_beg_end = df['activity'].tolist()
    for index, state in enumerate(act_col_beg_end):
        if str(state).split()[-1] == 'begin':
            uni_list_beg.append(state.strip().split()[0])
        elif str(state).split()[-1] == 'end' and (str(state).split()[0] in uni_list_beg):
            uni_list_beg.remove(str(state).split()[0])
    uni_list = list(uni_list)
    uni_list.sort()
    sensorID = df['sensorID'].tolist()
    sensor_list = [0] * (len(cols)-1)
    for index, sensor in enumerate(cols[:-1]):
        sensor_list[index] = int(sensor in sensorID)
    return uni_list, uni_list_beg, sensor_list

In [6]:
cols = data_milan.sensorID.unique()
#print(list(cols))
cols = list(cols)
cols.append('Activities')
df_sensors = pd.DataFrame(columns = cols)
df_sensors.head(5)

uni_list_beg = []
startTime = data_milan.index[0]
stepTime = pd.Timedelta(seconds=60)
endTime = data_milan.index[-1]
count = 0
countc = 0
while startTime  <= endTime:
    df_15mins = data_milan.loc[startTime: startTime + stepTime]
    #print(df_15mins)
    startTime = startTime + stepTime
#     print(startTime)
    uni_list, uni_list_beg, sensor_list =  process_subset(df_15mins, uni_list_beg, cols)
    print("count: ", count)
    print("List of open activities:", uni_list_beg)
    print("List of activities:", uni_list)
    #print(list(cols))
    count+=1
    print("Sensor Feature:", sensor_list)
    if (all(v == 0 for v in sensor_list) and uni_list_beg) or uni_list:
        print("countc: ", countc)
        countc+=1
        if all(v == 0 for v in sensor_list):
            uni_list = list(uni_list_beg)
        list2append = uni_list
        sensor_list.append(list2append)
        sensor_series = pd.Series(sensor_list, index= df_sensors.columns)      
        print(list(sensor_series))
        df_sensors = df_sensors.append(sensor_series, ignore_index=True)
    elif (all(v == 0 for v in sensor_list) and not uni_list_beg):
        print("countc: ", countc)
        countc+=1
        sensor_list.append(['Other'])
        sensor_series = pd.Series(sensor_list, index= df_sensors.columns)      
        print(list(sensor_series))
        df_sensors = df_sensors.append(sensor_series, ignore_index=True)

count:  0
List of open activities: []
List of activities: ['Other']
Sensor Feature: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  0
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  1
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  2
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  3
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

count:  41
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  41
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  42
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  42
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  43
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  43
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  44
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

count:  69
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  69
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  70
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  70
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  71
List of open activities: []
List of activities: ['Other']
Sensor Feature: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  71
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  72
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

count:  98
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  98
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  99
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  99
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  100
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  100
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  101
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

count:  126
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  126
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  127
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  127
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  128
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  128
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  129
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

count:  810
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  810
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  811
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  811
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  812
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  812
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  813
List of open activities: ['Leave_Home']
List o

count:  845
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  845
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  846
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  846
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  847
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  847
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  848
List of open activities: ['Leave_Home']
List o

count:  873
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  873
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  874
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  874
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  875
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  875
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  876
List of open activities: ['Leave_Home']
List o

count:  899
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  899
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  900
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  900
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  901
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  901
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

count:  926
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  926
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  927
List of open activities: ['Kitchen_Activity']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  927
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  928
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  928
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitche

In [7]:
pd.set_option('display.max_rows', df_sensors.shape[0]+1)
df_sensors

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M018,M007,M027,M002,M001,D001,M004,D003,D002,Activities
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]


In [8]:
df_sensors.to_csv('dataset/milan/data_processed_test.csv', index = False)
df_unique_rows = df_sensors.iloc[:,:-1]
df_unique_rows = df_unique_rows.drop_duplicates()
df_unique_rows.reset_index(drop=True, inplace=True)
df_unique_rows

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M026,M018,M007,M027,M002,M001,D001,M004,D003,D002
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
dictSeries = df_unique_rows.to_dict('index')
dictSeries

{0: {'M017': 1,
  'M009': 1,
  'M019': 1,
  'M020': 0,
  'T001': 0,
  'T002': 0,
  'M016': 0,
  'M015': 0,
  'M022': 0,
  'M012': 0,
  'M003': 0,
  'M005': 0,
  'M006': 0,
  'M008': 0,
  'M011': 0,
  'M028': 0,
  'M021': 0,
  'M025': 0,
  'M013': 0,
  'M010': 0,
  'M023': 0,
  'M014': 0,
  'M026': 0,
  'M018': 0,
  'M007': 0,
  'M027': 0,
  'M002': 0,
  'M001': 0,
  'D001': 0,
  'M004': 0,
  'D003': 0,
  'D002': 0},
 1: {'M017': 0,
  'M009': 0,
  'M019': 0,
  'M020': 0,
  'T001': 0,
  'T002': 0,
  'M016': 0,
  'M015': 0,
  'M022': 0,
  'M012': 0,
  'M003': 0,
  'M005': 0,
  'M006': 0,
  'M008': 0,
  'M011': 0,
  'M028': 0,
  'M021': 0,
  'M025': 0,
  'M013': 0,
  'M010': 0,
  'M023': 0,
  'M014': 0,
  'M026': 0,
  'M018': 0,
  'M007': 0,
  'M027': 0,
  'M002': 0,
  'M001': 0,
  'D001': 0,
  'M004': 0,
  'D003': 0,
  'D002': 0},
 2: {'M017': 0,
  'M009': 0,
  'M019': 0,
  'M020': 1,
  'T001': 0,
  'T002': 0,
  'M016': 0,
  'M015': 0,
  'M022': 0,
  'M012': 0,
  'M003': 0,
  'M005': 0,
 

In [10]:
key_list = list(dictSeries.keys())
val_list = list(dictSeries.values())
col_label = df_sensors.apply(lambda row: key_list[val_list.index(pd.Series(row[:-1]).to_dict())], axis = 1)
col_label = col_label.rename("Sensor_label")
df_sensors = df_sensors.join(col_label)
df_sensors

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M007,M027,M002,M001,D001,M004,D003,D002,Activities,Sensor_label
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],2
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,[Other],1


In [11]:
unique_data = [list(x) for x in set(tuple(x) for x in list(df_sensors['Activities']))]
dictSeries = {}
for index,ele in enumerate(unique_data):
    dictSeries[index] = ele

In [12]:
key_list = list(dictSeries.keys())
val_list = list(dictSeries.values())
col_label = df_sensors['Activities'].map(lambda row: key_list[val_list.index(row)])
col_label = col_label.rename("activity_label")
df_sensors = df_sensors.join(col_label)

In [18]:
df_sensors

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M027,M002,M001,D001,M004,D003,D002,Activities,Sensor_label,activity_label
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],0,13
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],2,13
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,[Other],1,13
